In [1]:
user_input = "تاپ زنانه سرژه مدل رنگ صورتی"

In [2]:
import pysolr
import pandas as pd
from tqdm import tqdm
from fuzzywuzzy import fuzz

/home/user/.local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
def get_dkp_id(url):
    try:
        chunks = url.split("/")

        for chunk in chunks:
            if chunk.startswith("dkp-"):
                section = chunk.split("-")
                return int(section[1])

        return "The entered link is not related to a product."

    except:
        return "An Error has Occurred"

In [ ]:
data = pd.read_csv("/home/user/Desktop/data - 05.csv", header=None)

In [ ]:
dkp_names = list(data[1])
dkp_ids = list(data[0])

dkp_ids = [get_dkp_id(x) for x in dkp_ids]

In [ ]:
solr = pysolr.Solr('http://localhost:8983/solr/Digikala', always_commit=True)

In [ ]:
solr.ping()

In [ ]:
for i in tqdm(range(len(dkp_ids[:10000]))):
    solr.add(   
        {
            "dkp_id": dkp_ids[i],
            "dkp_name": dkp_names[i],
        }
    )

In [155]:
result = solr.search(q=f'dkp_name:روسری زنانه مدل B022', rows=100, wt="python", sort="score asc")

In [156]:
dkp_names_result = []

In [158]:
for i in result:
    dkp_names_result.append(i["dkp_name"][0])
    print(i["dkp_name"][0])

روسری زنانه مدل نخ پاییزه دست دوز  کد 2155
روسری زنانه مدل N-Shokofeh 120
روسری زنانه مدل 01 کد 1104
روسری زنانه مدل 01 کد 1108
روسری زنانه مدل N-100
روسری زنانه مدل مجلسی 03
روسری زنانه مدل Blazhia 02
روسری زنانه مدل Blazhia 03
روسری زنانه مدل Blzhia 04
روسری زنانه مدل موهر 7010
روسری زنانه کد 1106
روسری زنانه مدل A001
روسری زنانه مدل B022
روسری زنانه مدل Blzhia01
روسری زنانه مدل C003
روسری زنانه مدل بابونه
روسری زنانه مدل D004
روسری زنانه مدل F05
روسری زنانه مدل G007
روسری زنانه مدل روسری آبرنگی


In [180]:
def Levenshtein(product, dkp_names, dkp_ids):

    max_score = 0
    result = ""

    for i in range(len(dkp_names)):

        similarity_score = fuzz.ratio(user_input, dkp_names[i])

        # print(f"{similarity_score} : {item}")

        if similarity_score >= max_score:
            max_score = similarity_score
            result = dkp_names[i] 
            id = dkp_ids[i]           

    return id, result

In [186]:
last_result = Levenshtein(user_input, dkp_names_result)
last_result

TypeError: Levenshtein() missing 1 required positional argument: 'dkp_ids'

In [181]:
def find_dkp_link(product: str):

    solr = pysolr.Solr('http://localhost:8983/solr/Digikala', always_commit=True)
    result = solr.search(q=f'dkp_name:{product}', rows=100, wt="python", sort="score asc")
    
    dkp_names = []
    dkp_ids = []

    for i in result:
        dkp_names.append(i["dkp_name"][0])
        dkp_ids.append(i["dkp_id"][0])
    
    last_result = Levenshtein(product, dkp_names, dkp_ids)
    
    return last_result

In [187]:
id, name = find_dkp_link("روسری زنانه مدل نخ پاییزه دست دوز")
print(f"{id} | {name}")

10000283 | روسری زنانه مدل روسری آبرنگی
